In [31]:
from connector.mongo_connector import MongoConnector
from haversine import haversine, Unit
import pandas as pd
from datetime import datetime

conn = MongoConnector()

You are connected to the database: mongo_db
-----------------------------------------------



#### Task 1
How many users, activities and trackpoints are there in the dataset (after it is inserted into the database).

The following code snippets count the number of users, activites, and track points recorded in the database. We note that the count is the same as the ones found in the previous assignment.

In [28]:
def task_1(connection: MongoConnector): 
    users_count = connection.db["User"].count_documents({})
    activities_count = connection.db["Activity"].count_documents({})
    trackpoints_count  = connection.db["TrackPoint"].count_documents({})
    print(f'Number of users: {users_count}')
    print(f'Number of activities: {activities_count}')
    print(f'Number of trackpoints: {trackpoints_count}')
    
task_1(conn)

Number of users: 173
Number of activities: 16048
Number of trackpoints: 9555437


#### Task 2
Find the average number of activities per user.

THe following code snippets finds the average number of activities performed. We do that by calculating the ratio of activities to the number of users.

In [29]:
def task_2(connection: MongoConnector): 
    users = connection.db["User"].count_documents({})
    activities = connection.db["Activity"].count_documents({})
    avg_numb = activities/users
    print(f"Average activity count per user: {round(avg_numb,2)}")

task_2(conn)

Average activity count per user: 92.76


#### Task 3
*Find the top 20 users with the highest number of activities.*

The following code snippets finds the 20 users with the highest number of activities performed.

In [30]:
def task_3(connection: MongoConnector): 
    pipeline = [
         {"$unwind": "$activities"},
    {
        "$group": {
            "_id": "$_id",
            "activity_count": { "$sum": 1 }
        }
    },
    {
        "$sort": {"activity_count": -1}
    },
    {
        "$limit": 20
    }
    ]

    users = connection.db["User"].aggregate(pipeline)
    for doc in users: 
        print(f'User: {doc["_id"]}, Activity_count: {doc["activity_count"]} ')

task_3(conn)

User: 128, Activity_count: 2102 
User: 153, Activity_count: 1793 
User: 025, Activity_count: 715 
User: 163, Activity_count: 704 
User: 062, Activity_count: 691 
User: 144, Activity_count: 563 
User: 041, Activity_count: 399 
User: 085, Activity_count: 364 
User: 004, Activity_count: 346 
User: 140, Activity_count: 345 
User: 167, Activity_count: 320 
User: 068, Activity_count: 280 
User: 017, Activity_count: 265 
User: 003, Activity_count: 261 
User: 014, Activity_count: 236 
User: 126, Activity_count: 215 
User: 030, Activity_count: 210 
User: 112, Activity_count: 208 
User: 011, Activity_count: 201 
User: 039, Activity_count: 198 


#### Task 4
*Find all users who have taken a taxi.*

The following code snippet finds all users that have taken a taxi.

In [32]:
def task_4(connection: MongoConnector):
    unique_user_ids = set()
    activities = connection.db["Activity"].find({"transportation_mode": "taxi"})
    for doc in activities: 
        user_id = doc["user_id"]
        trans_mode = doc["transportation_mode"]

        if user_id not in unique_user_ids: 
            print(f'User: {user_id}, Transportation_mode: {trans_mode}')
            unique_user_ids.add(user_id)

task_4(conn)

User: 010, Transportation_mode: taxi
User: 058, Transportation_mode: taxi
User: 062, Transportation_mode: taxi
User: 078, Transportation_mode: taxi
User: 080, Transportation_mode: taxi
User: 085, Transportation_mode: taxi
User: 098, Transportation_mode: taxi
User: 111, Transportation_mode: taxi
User: 128, Transportation_mode: taxi
User: 163, Transportation_mode: taxi


#### Task 5
*Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.*

The following code snippet counts the number of activities tagged with each transportation mode. It also excludes all activities without transportation modes.

In [34]:
def task_5(connection: MongoConnector): 
    pipeline = [
        {
            "$match": {
                "transportation_mode": {"$ne": None}
            }
        }, 
        {
            "$group": {
                "_id": "$transportation_mode", 
                "count": {"$sum": 1}
            }
        }, 
        {
            "$sort": {"count": -1}
        }
        
    ]

    trans_modes = connection.db["Activity"].aggregate(pipeline)

    for doc in trans_modes:
        print(f'Transport mode: {doc["_id"]}, Count: {doc["count"]}') 

task_5(conn)

Transport mode: walk, Count: 480
Transport mode: car, Count: 419
Transport mode: bike, Count: 263
Transport mode: bus, Count: 199
Transport mode: subway, Count: 133
Transport mode: taxi, Count: 37
Transport mode: airplane, Count: 3
Transport mode: train, Count: 2
Transport mode: run, Count: 1
Transport mode: boat, Count: 1


#### Task 6a)
*Find the year with the most activities.*

The following code snippet counts the number of activities within each year. We are here assuming that we count the activity based on their start_date_time. Sorting the resulting output, results in 2008 being the year with the greatest number of activities.


In [36]:
def task_6a(connection: MongoConnector): 
    pipeline = [
        {
            "$project": {
                "start_year": {"$year": "$start_date_time"}
            }
        }, 
        {
            "$group": {
                "_id": "$start_year", 
                "count": {"$sum" : 1}
            }
        }, 
        {
            "$sort": {"count": -1}
        }
    ]

    activity_year_count = connection.db["Activity"].aggregate(pipeline)
    for year in activity_year_count: 
        print(f'Year: {year["_id"]}, num_activites: {year["count"]}')

task_6a(conn)

Year: 2008, num_activites: 5895
Year: 2009, num_activites: 5879
Year: 2010, num_activites: 1487
Year: 2011, num_activites: 1204
Year: 2007, num_activites: 994
Year: 2012, num_activites: 588
Year: 2000, num_activites: 1


#### Task 6b)
*Is this also the year with most recorded hours?*

The following code snippet calculates the accumulated number of hours in activities performed within each year. We here assume that the hour duration of an activity is determined by the difference between the start and end time for each activity. Different from the above answer, 2009 is the year with the largest number of hours logged.

In [37]:
def task_6b(connection: MongoConnector): 
    time_diff = {
        "$project": {
            "start_year": {"$year": "$start_date_time"},
            "duration": {
                "$divide": [
                    {"$dateDiff": {
                        "startDate": "$start_date_time",
                        "endDate": "$end_date_time",
                        "unit": "second"
                    }},
                    3600
                ]
            }
        }
    }

    pipeline = [
        time_diff,
        {
            "$group": {
                "_id": "$start_year",
                "hours_count": {"$sum": "$duration"}
            }
        },
        {
            "$sort": {"hours_count": -1}
        }
    ]

    activity_year_durations = connection.db["Activity"].aggregate(pipeline)
    for year in activity_year_durations: 
        print(f'Year: {year["_id"]}, Recorded hours: {year["hours_count"]}')

task_6b(conn)

Year: 2009, Recorded hours: 11612.423888888889
Year: 2008, Recorded hours: 9200.797222222222
Year: 2007, Recorded hours: 2315.418611111111
Year: 2010, Recorded hours: 1388.7275
Year: 2011, Recorded hours: 1132.3516666666667
Year: 2012, Recorded hours: 711.2133333333334
Year: 2000, Recorded hours: 0.051111111111111114


#### Task 7
*Find the total distance (in km) walked in 2008, by user with id=112. We are here assuming that the distance walked should only be considered within single activities.*

The following code snippets calculate the total distance walked by user 112 in 2008. We note that we only count distance walked within each distinct activities, which are summed up in the end.

In [41]:


def haversine_distance(point1, point2): 
    return haversine(point1, point2, unit=Unit.KILOMETERS)

def extract_tp_info(connection: MongoConnector): 
    start_date_time = datetime(2008, 1, 1, 0, 0, 0)
    end_date_time = datetime(2008, 12, 31, 23, 59, 59)

    tp = connection.db["TrackPoint"].aggregate([{
        "$match": {
            "user_id": "112",
            "transportation_mode": "walk",
            "date_time": {
                "$gte": start_date_time, 
                "$lte": end_date_time
            }
        }
    }])

    return tp
    

def task_7(connection: MongoConnector): 
    track_points_cursor = extract_tp_info(connection)
    track_points = list(track_points_cursor)

    total_distance = 0

    for i in range(len(track_points) -1): 
        coord1 = track_points[i]["location"]["coordinates"]
        coord2 = track_points[i+1]["location"]["coordinates"]
        activity_id_1 = track_points[i]["activity_id"]
        activity_id_2 = track_points[i+1]["activity_id"]

        if activity_id_1 == activity_id_2: 
            pos1 = [(coord1[1], coord1[0])]
            pos2 = [(coord2[1], coord2[0])]
            distance = haversine_distance(pos1[0], pos2[0])
            total_distance += distance
    
    print(f'Distance walked by user 112 in 2008 in total:  {total_distance} km')

task_7(conn)

Distance walked by user 112 in 2008 in total:  112.32978595682037 km


#### Task 8
*Find the top 20 users who have gained the most altitude meters.*

The following code snippets calculate the top 20 users who have gained the most altitude meters across all track points in the dataset. Track points that have altitude value set to -777 are excluded, per the requirements. It is understood that only altitude gained should be included in the calculation. Thus, if the user descended in altitude between two subsequent track points, the descent is not included in the calculation.

We note that the resulting alttiude numbers are slightly different from that gained in the previous assignment. This is due to altitude being ingested to the MySQL database as INT, while the float datatype in the original dataset is preserved for the mongodb assignment. We feel like this is most appropiate, as it preserves the greatest level of data granularity. Furthermore, we have also verified that the results would have otherwhise been the same. P.s. INT was chosen for the MySQL assignment per the proposed schema.

In [40]:
def task_8(connection: MongoConnector):
    pipeline = [
        {
            "$match": {
                "altitude": {"$ne": -777}
            }
        }, 
         {
            "$sort": {
                "user_id": 1,
                "activity_id": 1,
                "date_time": 1
            }
        },
        {
            "$project": {
                "user_id": 1,
                "activity_id": 1,
                "altitude": 1,
                "date_time": 1
            }
        }
    ]

    output = list(connection.db["TrackPoint"].aggregate(pipeline))

    altitudes = {}
    for i in range(len(output) - 1): 
        tp = output[i]
        next_tp = output[i+1]
        if tp["activity_id"] == next_tp["activity_id"] and tp["user_id"] == next_tp["user_id"]:
            if tp["altitude"] < next_tp["altitude"]: 
                altitude_difference_meters = (next_tp["altitude"] - tp["altitude"]) * 0.3048
                altitudes.setdefault(tp["user_id"], 0)
                altitudes[tp["user_id"]] += altitude_difference_meters
    
    user_with_most_altitude_meters = sorted(altitudes.items(), key = lambda item: item[1], reverse=True)

    for a in user_with_most_altitude_meters[:20]: 
        print(f'User_id:  {a[0]}, total_altitude_meters_gained: {a[1]}')

task_8(conn)

User_id:  128, total_altitude_meters_gained: 615465.6600795912
User_id:  153, total_altitude_meters_gained: 495868.20985812164
User_id:  004, total_altitude_meters_gained: 294546.832800116
User_id:  041, total_altitude_meters_gained: 227222.79167998934
User_id:  003, total_altitude_meters_gained: 200865.3336000703
User_id:  085, total_altitude_meters_gained: 196490.81352013102
User_id:  163, total_altitude_meters_gained: 190135.57305897324
User_id:  144, total_altitude_meters_gained: 175432.6203200511
User_id:  030, total_altitude_meters_gained: 154214.16960000098
User_id:  062, total_altitude_meters_gained: 145087.23839997425
User_id:  039, total_altitude_meters_gained: 136360.50959998736
User_id:  084, total_altitude_meters_gained: 122486.31839999456
User_id:  002, total_altitude_meters_gained: 106269.73919997527
User_id:  000, total_altitude_meters_gained: 105622.64879999145
User_id:  167, total_altitude_meters_gained: 104015.24263999812
User_id:  025, total_altitude_meters_gained: 

#### Task 9 
*Find all users who have invalid activities, and the number of invalid activities per user.*

The following code snippet computes the number of invalid activities for each user by taking the time difference between subsequent points, and counting the activity as invalid if the time difference is greater than or equals to 5 minutes. We note that the answer is exactly the same as the one attained in the previous assignment.

In [69]:
def task_9(connection: MongoConnector): 
    pipeline = [
        {
            "$match": {
                "prev_date_time": {"$ne": None}
            }
        },
        {
            "$project": {
                "user_id": 1,
                "activity_id": 1,
                "time_difference": {
                    "$divide": [
                    {
                        "$subtract": ["$date_time", "$prev_date_time"]
                    },
                    60 * 1000
                    ]
                }
            }
        }, 
        {
            "$match": {
                "time_difference": {"$gte": 5}
            }
        },
        {
            "$group": {
                "_id": {"user_id": "$user_id", "activity_id": "$activity_id"},
                "invalid": {"$max": 1}
            }
        },
        {
            "$group": {
                "_id": "$_id.user_id", 
                "invalid_activity_count": {"$sum": 1}
            }
        },
        {
            "$sort": {"invalid_activity_count": -1}
        }
    ]

    invalid_activities = connection.db["TrackPoint"].aggregate(pipeline)
    
    for activity in invalid_activities: 
        print(f'User_id: {activity["_id"]}, Invalid_activities_number: {activity["invalid_activity_count"]}')

task_9(conn)

User_id: 128, Invalid_activities_number: 728
User_id: 153, Invalid_activities_number: 549
User_id: 025, Invalid_activities_number: 266
User_id: 062, Invalid_activities_number: 251
User_id: 163, Invalid_activities_number: 233
User_id: 004, Invalid_activities_number: 220
User_id: 041, Invalid_activities_number: 198
User_id: 085, Invalid_activities_number: 183
User_id: 003, Invalid_activities_number: 181
User_id: 144, Invalid_activities_number: 158
User_id: 039, Invalid_activities_number: 147
User_id: 068, Invalid_activities_number: 140
User_id: 167, Invalid_activities_number: 138
User_id: 017, Invalid_activities_number: 125
User_id: 014, Invalid_activities_number: 120
User_id: 030, Invalid_activities_number: 113
User_id: 092, Invalid_activities_number: 109
User_id: 126, Invalid_activities_number: 105
User_id: 000, Invalid_activities_number: 102
User_id: 037, Invalid_activities_number: 100
User_id: 002, Invalid_activities_number: 98
User_id: 104, Invalid_activities_number: 98
User_id: 084

#### Task 10
*Find the users who have tracked an activity in the Forbidden City of Beijing.*

The following code snippets compute all the users that have been within approx. 1km radius of the given coordinates given for the forbidden city. If only the exact point is considered, then we would have received a match for user 18 and 19.

In [75]:
def task_10(connection: MongoConnector): 
    pipeline = [
        {
            "$match": {
                "location": {
                    "$geoWithin": {
                        "$centerSphere": [
                            [116.397, 39.916], 1 / 6371.0
                        ]
                    }
                }
            }
        },
        {
            "$group": {
                "_id": "$user_id"
            }
        }
    ]
    
    print("Users that have been in forbidden city are: ") 
    user_match = connection.db["TrackPoint"].aggregate(pipeline)
    for user in user_match:
        print(f'User: {user["_id"]}')

task_10(conn)

Users that have been in forbidden city are: 
User: 112
User: 169
User: 140
User: 018
User: 067
User: 084
User: 004
User: 136
User: 144
User: 019
User: 102
User: 082
User: 051
User: 025
User: 068
User: 135
User: 041
User: 168
User: 153
User: 128
User: 163
User: 062
User: 052
User: 119
User: 131
User: 034


#### Task 11
*Find all users who have registered transportation_mode and their most used transportation_mode.*

We first group by both user_id and transportation_mode to count occurrences for each mode per user. We then sort the results by user_id and count in descending order. 
After that, we use another $group stage to select the first (most used) mode for each user. Finally, we sort the results by user_id.
This approach ensures that we get the most used mode for each user, and if there are ties, it selects the first one based on the sort order.

In [71]:
def task_11(connection: MongoConnector): 
    pipeline = [
        {
            "$match": {
                "transportation_mode": {"$ne": None}
            }
        }, 
        {
            "$group": {
                "_id": {"user_id": "$user_id", "trans_mode": "$transportation_mode"},
                "count": {"$sum" : 1}
            }
        }, 
        {
            "$sort": {"_id.user_id": 1, "count": -1}
        },
        {
            "$group": {
                "_id": "$_id.user_id", 
                "most_used_trans_mode": {"$first": "$_id.trans_mode"},
                "count": {"$first": "$count"}
            }
        }, 
        {
            "$sort": {"_id": 1}
        }
    ]

    user_most_used_trans_mode = connection.db["Activity"].aggregate(pipeline)
    for doc in user_most_used_trans_mode: 
        user_id = doc["_id"]
        trans_mode = doc["most_used_trans_mode"]
        count = doc["count"]
        print(f'User: {user_id}, Most used tran_mode is: {trans_mode} with count {count}')

task_11(conn)

User: 010, Most used tran_mode is: taxi with count 3
User: 020, Most used tran_mode is: bike with count 81
User: 021, Most used tran_mode is: walk with count 1
User: 052, Most used tran_mode is: bus with count 1
User: 056, Most used tran_mode is: bike with count 15
User: 058, Most used tran_mode is: car with count 2
User: 060, Most used tran_mode is: walk with count 1
User: 062, Most used tran_mode is: walk with count 173
User: 064, Most used tran_mode is: bike with count 1
User: 065, Most used tran_mode is: bike with count 10
User: 067, Most used tran_mode is: walk with count 1
User: 069, Most used tran_mode is: bike with count 1
User: 073, Most used tran_mode is: walk with count 52
User: 075, Most used tran_mode is: walk with count 1
User: 076, Most used tran_mode is: car with count 3
User: 078, Most used tran_mode is: walk with count 37
User: 080, Most used tran_mode is: bike with count 1
User: 081, Most used tran_mode is: bike with count 4
User: 082, Most used tran_mode is: walk wi